In [1]:
import numpy as np
from os import path

Functions for loading data

In [2]:
def load_labels(folder, feature_column):
  y = np.genfromtxt(path.join(folder,'labels.csv'),
    dtype=np.int8,
    usecols=(feature_column),
    delimiter='\t',
    skip_header=1
  )
  return y

Test load labels

In [4]:
DATASET = 'Datasets/celeba'
FEATURE = 2 # column 2 contains gender
print(load_labels(DATASET, FEATURE))

[-1 -1  1 ...  1  1  1]
